<center><img src="http://i.imgur.com/sSaOozN.png" width="500"></center>

## Course: Computational Thinking for Governance Analytics

### Prof. José Manuel Magallanes, PhD 
* Visiting Professor of Computational Policy at Evans School of Public Policy and Governance, and eScience Institute Senior Data Science Fellow, University of Washington.
* Professor of Government and Political Methodology, Pontificia Universidad Católica del Perú. 

_____

# Session 2:  Data Preprocessing in Python

<a id='beginning'></a>

Preprocessing includes three stages:

1. [Collecting](#part1) 
2. [Cleaning](#part2) 
3. [Formatting](#part3) 

____
<a id='part1'></a>

## Collecting

Collection is not a pre processing problem if you have data already nicely organized. When it is so, **pandas** will read the file without problem:

* Reading STATA file

In [1]:
import pandas as pd

stataFile='https://github.com/EvansDataScience/data/raw/master/lapopUSA2017_13.dta'
##
dataStata=pd.read_stata(stataFile,convert_categoricals=False)

* Reading EXCEL file

In [2]:
excelFile='https://github.com/EvansDataScience/data/raw/master/hdi2016.xlsx'
dataExcel=pd.read_excel(excelFile)

As you just have seen, a common file type can be easily read using pandas. Once you have them, you can ask many things, some could be:

* Number of rows and columns:

In [3]:
dataStata.shape

(1500, 119)

* See the first rows:

In [6]:
dataExcel.head()

Country  Human Development Index (HDI)   Life expectancy at birth  \
0       Norway                        0.949423                    81.711   
1    Australia                        0.938680                    82.537   
2  Switzerland                        0.939131                    83.133   
3      Germany                        0.925669                    81.092   
4      Denmark                        0.924649                    80.412   

   Expected years of schooling   Mean years of schooling  \
0                      17.67187                12.746420   
1                      20.43272                13.175100   
2                      16.04041                13.370000   
3                      17.09594                13.187626   
4                      19.18880                12.700170   

   Gross national income (GNI) per capita                     Category  
0                             67614.35348  VERY HIGH HUMAN DEVELOPMENT  
1                             42822.19627  VERY HIGH HUMAN DEVELOPMENT  
2                             56363.95780  VERY HIGH HUMAN DEVELOPMENT  
3                             44999.64714  VERY HIGH HUMAN DEVELOPMENT  
4                             44518.92402  VERY HIGH HUMAN DEVELOPMENT

The property __shape__ is an attribute of the data frame, so you do not need parentheses; you do not these ones to call a function, in this last case __head()__. You also have __tail()__:

In [ ]:
dataStata[['q12','q12m','q12f']].tail()

The tail is showing only the last elements of the selected columns.

* Self-collecting

Another way to collect data is to create an online form and then get the data into Python. For this situation, go to this [link](https://goo.gl/forms/HX3KkxcEtXgMzyDJ3) and answer the questions.

Then, I will see the answers:


In [ ]:
#link='write here'
#namesOfCols=['timeStamp','name','sex','age','bornIn','workingStatus']

#myData = pd.read_csv(link,header=0,names=namesOfCols)

# here it is:
#myData.head(10)

The information collected was retrieved in a comma-separated values format. This is a very common format. Notice the settings of __read_csv__: I tell pandas that the first row has the names of the columns (header is in position 0); and then I rename the headers.

* Collecting data from APIs

Most of these data come in more complex formats, like XML or JSON format. Let's get the data about 
[Seattle Real Time Fire 911 Calls](https://dev.socrata.com/foundry/data.seattle.gov/grwu-wqtk). Let me follow the instructions from that website to get the data:

In [ ]:
# pip install sodapy

from sodapy import Socrata

client = Socrata("data.seattle.gov", None)
results = client.get("grwu-wqtk", limit=1000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df.head()

[Go to page beginning](#beginning)

_____
<a id='part2'></a>

## Cleaning

The files opened above look clean because they may have been produce professionally for statistical work, or because the collecting tool was very restrictive. However, several data that you collect may not bring you the data as clean as the ones found in the previous codes. This commmon webpage has a table that may be needed:

In [ ]:
wikiLink="https://en.wikipedia.org/wiki/List_of_freedom_indices"


import IPython
iframe = '<iframe src=' + wikiLink + ' width=700 height=350></iframe>'
IPython.display.HTML(iframe)

Let's try to get the table using pandas:

In [ ]:
import pandas as pd

wikiTables=pd.read_html(wikiLink,header=0,attrs={'class': 'wikitable sortable'})

I tried to get all the tables. I may have more than one:

In [ ]:
# What do I have? / How many?
type(wikiTables), len(wikiTables) 

I need to recover the first table from the list (the only one).

In [ ]:
DF=wikiTables[0]

#what is it?
type(DF)

Great!...we have a data frame; then:

In [ ]:
DF.head()

This data frame does not look like the one we see on the website. We need to improve the call:

In [ ]:
# install 'beautifulsoup4'
DF=pd.read_html(wikiLink,header=0,flavor='bs4',attrs={'class': 'wikitable sortable'})[0]
DF.head()

Combining BeautifulSoup (BS) and Pandas gave us the right result. But our work is not over.

Pay attention to the cleaning pandas+BS have done: the 'n/a' was interpreted as **NaN**; no country flags in the data; and the headers are in the right place. 

However, to prepare a final data set, we should pay attention to the headers names to avoid _blanks_, and erase the _footnote_ call.

We can have two strategies:
* Brute-force!

In [ ]:
# if we had a small number of names to change, we can use brute-force strategy:
DF.columns=['Country',
 'FreedomintheWorld',
 'IndexofEconomicFreedom',
 'PressFreedomIndex',
 'DemocracyIndex']
DF.head()

* Using more computational thinking (algorithmic):

In [ ]:
# if we had many columns, writing an algorith to rename the columns could be better:

# recalling the data:
DF=pd.read_html(wikiLink,header=0,flavor='bs4',attrs={'class': 'wikitable sortable'})[0]

I just recalled the data to do several steps:

1. Find blanks.
2. Find numbers.
3. Find brackets (opening and closing).

The previous requires a **regular expresssion**:

In [ ]:
import re  # may need to be installed:

# find blanks: \\s+
# find numbers: \\d+
# find opening bracket : \\[
# find closing bracket: \\]

# You can combine using '|' (or):
pattern='\\s+|\\d+|\\[|\\]'
nothing=''

Now, let's see how this works for one case:

In [ ]:
testString='Freedom in the World 2018[10]'
re.sub(pattern,nothing,testString)

Now, let's see how this works for ALL cases:

In [ ]:
[re.sub(pattern,nothing,name) for name in DF.columns]

We can verify we are matching well:

In [ ]:
newNames=[re.sub(pattern,nothing,name) for name in DF.columns]

# checking:
list(zip(DF.columns,newNames))

Let's turn that match into a dictionary:

In [ ]:
{old:new for old,new in zip(DF.columns,newNames)}

Once you have a dict like that one, you can use it to rename the columns with another function:

In [ ]:
changes={old:new for old,new in zip(DF.columns,newNames)}
DF.rename(columns=changes,inplace=True)

If you had a set of new names, and you do not want to change every column name, that is the correct way to do it.

Let's see the result:

In [ ]:
DF.head()

A next step will be verifying if the answers are well coded:

In [ ]:
DF.iloc[:,1::].describe()

What were you looking for? 
Sometimes a category may be wrongly written in a cell, for instance, if you had 'Free' and 'free' or 'free ' to represent the same in one column, you have a mistake. Let's see if there is one here:

In [ ]:
DF.FreedomintheWorld.value_counts()

 What we see is that this variable has its own correct set of answers. 
 
 We can try that approach for each variable, but we can check the whole group of categorical values like thisL

In [ ]:
# DF.iloc[:,1::] all columns but the first one
# apply(set)  apply the function 'set()'  per column
# tolist() convert to a list 

DF.iloc[:,1::].apply(set).tolist()

[Go to page beginning](#beginning)
____
<a id='part3'></a>
## Formatting

The data seems _clean_, but we need now to be sure the information is in the right format. This varies according to the project; so, let me show you some steps during of the formatting stage.

1. Verify the data types:


In [ ]:
DF.dtypes

All but the first variable are categories, not text (_object_). To convert them into categories you can do this:

In [ ]:
headerNames=DF.columns
DF[headerNames[1:]]=DF[headerNames[1:]].astype('category')

When a variable is of categorical type, you can use particular functions for them:

In [ ]:
DF.FreedomintheWorld.cat.categories

In [ ]:
DF.IndexofEconomicFreedom.cat.categories

In [ ]:
DF.PressFreedomIndex.cat.categories

In [ ]:
DF.DemocracyIndex.cat.categories

2. If ordinal, make the adjustment.

The order in which the categories differentiate a plain categorical from an ordinal categorical. They should be categorical but the order does not reflect the order it should. 

We can turn it into an ordinal doing the following:

a. Find a good numeric sequence for the ordinal values:

In [ ]:
# notice I am using the numbers in the same order as the list of categorical values:
oldFree=list(DF.FreedomintheWorld.cat.categories)

# '5 very good' / '4 good' / '3 middle' / '2 bad' / '1 very bad'

newFree=[5,1,3]
recodeFree={old:new for old,new in zip (oldFree,newFree)}

oldEco=list(DF.IndexofEconomicFreedom.cat.categories)
newEco=[5,3,4,2,1]
recodeEco={old:new for old,new in zip (oldEco,newEco)}

oldPress=list(DF.PressFreedomIndex.cat.categories)
newPress=[2,5,3,4,1]
recodePress={old:new for old,new in zip (oldPress,newPress)}

oldDemo=list(DF.DemocracyIndex.cat.categories)
newDemo=[1,4,5,2]
recodeDemo={old:new for old,new in zip (oldDemo,newDemo)}

b. Rename the still plain categorical:

In [ ]:
DF.FreedomintheWorld.cat.rename_categories(recodeFree,inplace=True)

DF.IndexofEconomicFreedom.cat.rename_categories(recodeEco,inplace=True)

DF.PressFreedomIndex.cat.rename_categories(recodePress,inplace=True)

DF.DemocracyIndex.cat.rename_categories(recodeDemo,inplace=True)

# veamos:
DF.head(10)

c. Now turn the renamed columns into a numeric values:

In [ ]:
DF[headerNames[1:]]=DF[headerNames[1:]].apply(pd.to_numeric)

Let me verify:

In [ ]:
DF.head()

3. Try solving missing data presence

The data has some missing data:

In [ ]:
DF.info()

Now comes the thinking: How to replace the missing values?

Python can easily find and replace every missing value; but our strategy will be different:

* _Freedom in the World_ has the least missing values, we will use this variable to see how the others behave.

* Since the variables are ordinals (even though they are numbers now) a good candidate to impute a missing is the median NOT the mean (you can not compute the mean of an ordinal).

Let's see:

In [ ]:
#median per group: 
DF.groupby('FreedomintheWorld')[headerNames[2:]].median()

We need to replace those medians whenever a missing value is found:

In [ ]:
for col in headerNames[2:]:
    # in each column, get median by FIW group, and use it to replace the missing values.
    DF[col].fillna(DF.groupby(["FreedomintheWorld"])[col].transform("median"), inplace=True)

In [ ]:
DF.head(20)

We can send this to R, in a simple CSV format:

In [ ]:
#DF.to_csv("indexes.csv",index=None)

______

## More examples

### Case: Democracy Index

Let me clean a similar data from wikipedia, about democracy index:

In [ ]:
import pandas as pd #location:
demoLink = "https://en.wikipedia.org/wiki/Democracy_Index" 

#collection
demodex=pd.read_html(demoLink,header=0,flavor='bs4',attrs={'class': 'wikitable sortable'})[0]

1. Looking for messiness:

In [ ]:
# what's on top?
# names? weird symbols? more links?
demodex.head(10)

In [ ]:
# what's at the bottom?
# note? credits? extra info?

demodex.tail(10)

First, we see a column that have some messiness (symbol "=" in rank), but which can be deleted as their information is not relevant. Let me get rid of the _Score_, as it is just the mean of the other ones. The last row is the repetition of the headers, so that one should go, too:

In [ ]:
#bye row 167, and two columns
demodexClean=demodex.drop(index=167,columns=['Rank','Score'])

In [ ]:
demodexClean

As there are few names, we can change to smaller sizes:

In [ ]:
newNames=['plularism','effectiveness','participation','culture','liberties']

# names from the second and before the last one '[1:-1]':
newMapper={old:new for old,new in zip(demodexClean.columns[1:-1],newNames)}

demodexClean.rename(columns=newMapper,inplace=True)

In [ ]:
# this is what we have so far:
demodexClean.head()

It looks good so far. Let's go to formatting.

2. Giving the rigth format:

In [ ]:
# checking data types:
demodexClean.dtypes

Above, we realized the need to make some indices into numeric:

In [ ]:
demodexClean[newNames]=demodexClean[newNames].apply(pd.to_numeric)

The last one is a categorical variable:

In [ ]:
demodexClean.Category.value_counts()

When you have text, you could get the unique values of a column like this:

In [ ]:
pd.unique(demodexClean.Category).tolist()

Then, you can prepare the map to recode the values:

In [ ]:
oldValues=pd.unique(demodexClean.Category).tolist()
newValues=[4,3,2,1]
mapNewOld={old:new for old,new in zip(oldValues,newValues)}
mapNewOld

You can do it in this way:

In [ ]:
demodexClean.Category.replace(mapNewOld,inplace=True)

In [ ]:
# or this one:
# demodexClean.Category=demodexClean.Category.replace(mapNewOld)

You can save it as a category, but that will be lost if sent to R:

In [ ]:
demodexClean.Category=demodexClean.Category.astype('category')

In [ ]:
demodexClean['Category'].cat.categories

In [ ]:
# checking missing values

demodexClean.info()

This data is now ready for R.

In [ ]:
# demodexClean.to_csv("democracyIndex.csv",index=None)

### The case of Medicare:

Here I will use data from [Medicare Beneficiary Enrollment and Demographics](https://dev.socrata.com/foundry/data.wa.gov/2cup-2fnu)

In [ ]:
import requests

# This time I am talking to the API from DATA.WA.GOV
url = "https://data.wa.gov/resource/2cup-2fnu.json?year=2014"
response = requests.get(url)
if response.status_code == 200:
    medicare = response.json()

In [ ]:
# turning json into DF:
medicare2014 = pd.DataFrame(medicare)

In [ ]:
medicare2014.head()

In [ ]:
medicare2014.tail()

The first row is the total, it has to go:

In [ ]:
#this one?
medicare2014.drop(index=0).head()

In [ ]:
#or this one?
medicare2014.drop(index=0).reset_index().head()

In [ ]:
#or this one?
medicare2014.drop(index=0).reset_index(drop=True).head()

When we use inplace, we should not concatenate:

In [ ]:
medicare2014.drop(index=0,inplace=True)
medicare2014.reset_index(drop=True,inplace=True)

The result so far:

In [ ]:
medicare2014.head()

In [ ]:
# what we have
medicare2014.dtypes

Notice that the three variables before the last one, and county should be kept as objects, while the other should be numeric:

In [ ]:
# get original order:
original=medicare2014.columns.tolist()
original

In [ ]:
# new order:  (no need for * if one element)
newOrder=[original[3],*original[14:], *original[0:3],*original[4:14],] # using '*'
newOrder

In [ ]:
# there are differenet data types, let me move columns:

medicare2014=medicare2014[newOrder]
medicare2014.head()

2. Formatting

We can give the right format now:

In [ ]:
headerNames=medicare2014.columns
medicare2014[headerNames[4::]]=medicare2014[headerNames[4::]].apply(pd.to_numeric)

In [ ]:
#check data types:
medicare2014.dtypes

We can explore the variables:

In [ ]:
medicare2014.describe(include='all') # to include categorical

In [ ]:
medicare2014.info()

There are some missing values, but we will leave it so. So the last step will be just to save the file:

In [ ]:
# medicare2014.to_csv("medicare2014.csv",index=None)

### Case: Public education:

When you visit the [website](https://nces.ed.gov/ccd/) of the Common Core of Data from the US Department of Education, you can get a data set with detailed information on public schools at the state of Washington:

In [ ]:
dataFile='https://github.com/EvansDataScience/data/raw/master/wapubs.xlsx'
schoolPub=pd.read_excel(dataFile) 

1. Looking for messiness:

In [ ]:
schoolPub.head(20)

The first row is not the beginning of the table. We need to skip 11 rows; but pay attention to what you are deleting, as if is telling you how missing values were coded.

In [ ]:
schoolPub=pd.read_excel(dataFile,skiprows=11,na_values=['†','‡','–'])
schoolPub.head()

In [ ]:
#checking the tail:
schoolPub.tail()

The headers have blanks and symbols, getting rid of them here:

In [ ]:
#import re  

pattern='\\*|\\s+'
nothing=''
schoolPub.columns=[re.sub(pattern,nothing,columnName) for columnName in schoolPub.columns]
schoolPub.columns

Clean names allow better exploring. Notice we solved the missing values above. You could have done this instead:

In [ ]:
#symbolsForNA=['†','‡','–'] 

#import numpy as np  #numpy manages the nan for pandas
#schoolPub.replace(symbolsForNA,np.nan,inplace=True) # in the whole data frame!!

2. Formatting

In [ ]:
schoolPub.dtypes

Even though we cleaned the missing values, there might be more in the text columns that may be hidden. Obviously, 'SchoolName','District','CountyName','StreetAddress','City','State' are text, but the other are possibly categorical.

So let me explore all the other ones, which are of type _object_:

In [ ]:
notUsed=['SchoolName','District','CountyName','StreetAddress','City','State']
 
# These are the ones without the obvious text columns
schoolPub.drop(notUsed,axis=1).head()

In [ ]:
# # These are the ones without the obvious text columns, but of the type 'object':
schoolPub.drop(notUsed,axis=1).select_dtypes(include='object').head()

We need to see the categories there:

In [ ]:
schoolPub.drop(notUsed,axis=1).select_dtypes(include='object').apply(set).tolist()

We need to take care of the missing value '**N**':

In [ ]:
schoolPub.Locale.value_counts(dropna=False)

Then:

In [ ]:
import numpy as np  #numpy manages the nan for pandas

schoolPub.replace(['N'],np.nan,inplace=True) # in the whole data frame!!

In [ ]:
# So:
schoolPub.Locale.value_counts(dropna=False)

Another important step could be to give add some text to make the school grades a recognizable ordering (considering the file will be read in R:

In [ ]:
# this is wrong:
'PK'<'KG'<'01'

In [ ]:
# this is OK:
'-1 PK'<'0 KG'<'01'

In [ ]:
# using replace:

schoolPub.replace({'PK':"-1 PK", "KG":"0 KG"},inplace=True)

Unless you want to recode other [variables](https://nces.ed.gov/programs/edge/docs/LOCALE_CLASSIFICATIONS.pdf), we could save this file:

In [ ]:
# schoolPub.to_csv("schoolPub.csv",index=None)

### Case: SNAP

In [ ]:
import pandas as pd
dataFile="https://github.com/EvansDataScience/data/raw/master/cntysnap.xls"
snapBen=pd.read_excel(dataFile)

In [ ]:
# first rows:
snapBen.head()

We need to skip some rows:

In [ ]:
# skipping:

snapBen=pd.read_excel(dataFile,skiprows=2)
snapBen.head()

In [ ]:
# check the tail
snapBen.tail()

In [ ]:
# checking names:
snapBen.columns

In [ ]:
# getting rid of blanks:

pattern='\\s+'
nothing=''
snapBen.columns=[re.sub(pattern,nothing,columnName) for columnName in snapBen.columns]

There is a zero FIPS code, take a look:

In [ ]:
snapBen[snapBen['CountyFIPScode']==0]

Those are rows about States. I will keep the counties:

In [ ]:
snapBenUSCounties=snapBen[snapBen['CountyFIPScode']!=0]

In [ ]:
# checking data types:
snapBenUSCounties.dtypes

The counties tell you to what State they belong, so we could use that to create a new column. Let's see a simple example on how to get information from a text:

In [ ]:
# using split,a function for strings:
'Autauga County, AL'.split(', ') # notice the space after the comma
# you get a list:

The **split**, in this case, returns the state in the second position of the list (index=1), then:

In [ ]:
# saving every second element for each element in the column:
states=[element.split(', ')[1] for element in snapBenUSCounties.Name]

# make that list a new column
snapBenUSCounties=snapBenUSCounties.assign(StateName=states)

# checking:
snapBenUSCounties

The new column was created. We could get rid of the state information from the counties column:

In [ ]:
# just keep county names
counties=[element.split(', ')[0] for element in snapBenUSCounties.Name]
snapBenUSCounties=snapBenUSCounties.assign(Name=counties)

In [ ]:
# quick look:

snapBenUSCounties.head() # last column will be ate the end...

We can have a better column order:

In [ ]:
oldNames=snapBenUSCounties.columns.tolist()
oldNames

In [ ]:
newNames=[*oldNames[:2],oldNames[-1],*oldNames[2:-1]]
newNames          

In [ ]:
# reordering columns:

snapBenUSCounties=snapBenUSCounties[newNames]
snapBenUSCounties.head()

In [ ]:
# JUST SAVING...
#snapBenUSCounties.to_csv("snapBenUSCounties.csv",index=None)

### Case: Multiple data sets

In [ ]:
corruptLink='https://raw.githubusercontent.com/EvansDataScience/data/master/corruption.csv'
econoLink='https://raw.githubusercontent.com/EvansDataScience/data/master/economic.csv'
enviroLink='https://raw.githubusercontent.com/EvansDataScience/data/master/environment.csv'
pressLink='https://raw.githubusercontent.com/EvansDataScience/data/master/pressfreedom.csv'

* The _corruptlink_ has data about the _Corruption Perception Index_ (CPI) produced by [Transparency International](https://www.transparency.org/).

* The _econoLink_ has data about the _Economic Freedom Index_ (EFI) produced by [Fraser Institute](https://www.fraserinstitute.org).

* The _enviroLink_ has data about the _Environment Performance Index_ (EPI) produced by [Yale University and Columbia University in collaboration with the World Economic Forum](https://epi.envirocenter.yale.edu/).

* The _pressLink_ has data about the _World Press Freedom Index_ (WPFI) produced by [Reporters Without Borders](https://rsf.org/en/world-press-freedom-index).


In this case, I want to join them (not concatenate):

In [ ]:
import pandas as pd
corrupt=pd.read_csv(corruptLink,encoding='Latin-1')
econo=pd.read_csv(econoLink,encoding='Latin-1')
enviro=pd.read_csv(enviroLink,encoding='Latin-1')
press=pd.read_csv(pressLink,encoding='Latin-1')

As each data set has a differing amount of rows (countries), and possibly a different way to name each one, the result will be far from perfect:

In [ ]:
join1=pd.merge(corrupt,econo)
join2=pd.merge(press,enviro)
indexes=pd.merge(join1,join2)

As always it is good to verify the data types:

In [ ]:
indexes.dtypes

And check descriptives:

In [ ]:
indexes.describe(include='all') 

In [ ]:
indexes.head()

There is some formatting needed:

Let's order it:

In [ ]:
oldCols=indexes.columns.tolist()
oldCols

When we do not have slices, there is extra work:

In [ ]:
numericIndex=[oldCols[i] for i in [1,3,4,6]]
numericIndex

In [ ]:
newValues=[oldCols[0],oldCols[2],*numericIndex,oldCols[5],oldCols[7]]
newValues

Then, the new order will be:

In [ ]:
indexes=indexes[newValues]
indexes.head()

There are several numeric values. Let's see a summary:

In [ ]:
indexes.describe()

It is important to find some monotony issues in these values:

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
pd.plotting.scatter_matrix(indexes.iloc[:,2:6])
plt.show()

Score press is negatively correlated to the rest. That means that the score for that column needs to be reversed:

In [ ]:
# creating reversing function:
def reverse(aColumn):
    return max(aColumn) - aColumn + min(aColumn)

In [ ]:
# reversing using function:
indexes.scorepress=reverse(indexes.scorepress)

We should see a different result:

In [ ]:
pd.plotting.scatter_matrix(indexes.iloc[:,2:6])
plt.show()

The variable _presscat_ needs to be an ordinal factor.

In [ ]:
indexes['presscat'].value_counts()

In [ ]:
indexes['presscat'].replace({'Medium':2, "High":3, "Low":1},inplace=True)

In [ ]:
indexes['presscat'].value_counts(sort=False)

The numbers will help R users when they set it as an ordinal. You can convert them to ordinal, but that information will be lost in R.

In [ ]:
indexes.head()

We are proposing that the categories coded as numbers follow an asceding format, then let's check if _environmentCat_ should be changed:

In [ ]:
indexes['environmentCat'].value_counts()

As there is no need for that, just save the file:

In [ ]:
# indexes.to_csv("indexes.csv",index=None)


____

* [Go to page beginning](#beginning)
* [Go to REPO in Github](https://github.com/EvansDataScience/ComputationalThinking_Gov_2)
* [Go to Course schedule](https://evansdatascience.github.io/GovernanceAnalytics/)